In [1]:
from memory_profiler import profile
import pennylane as qml
import pennylane.numpy as np
import time
from functools import wraps
from line_profiler import LineProfiler
import csv

In [2]:
# wires_m=[0]
# wires_solution=[1,2,3,4,5,6,7,8,9,10,11]

# Since the matrix is entirely comprised of the value "2" (see where val is
# defined), the input m only requires two qubits since 2 = 2^1 + 0 * 2^0.
# For larger numbers, use the other uncommented wire setups below (note that 
# using a setup with fewer qubits than is required will throw an error, like
# using the one qubit setup above for the value "2").

wires_m=[0,1]
wires_solution=[2,3,4,5,6,7,8,9,10,11,12]

# wires_m=[0,1,2]
# wires_solution=[3,4,5,6,7,8,9,10,11,12,13]
#
# wires_m=[0,1,2,3]
# wires_solution=[4,5,6,7,8,9,10,11,12,13,14]

# wires_m=[0,1,2,3,4]
# wires_solution=[5,6,7,8,9,10,11,12,13,14,15]

In [3]:
dev = qml.device("default.qubit", wires=wires_m  + wires_solution, shots=1)
# dev = qml.device("qiskit.ibmq", wires=wires_m + wires_solution, shots=1000, backend='',overwrite=True, ibmqx_token='')

n_wires = len(dev.wires)

In [ ]:
# Used to calculate the time spent on addition, multiplication, and in total
# for a single matrix multiplication. Time is returned in milliseconds.

add_time=0
mul_time=0
total_time=0

def calculate_time_add(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        execution_time = (end - start) * 1000  # 转换成毫秒
        global add_time
        add_time+=execution_time
        return result
    return wrapper

def calculate_time_mul(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        execution_time = (end - start) * 1000  # 转换成毫秒
        global mul_time
        mul_time+=execution_time
        return result
    return wrapper

def calculate_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        execution_time = (end - start) * 1000  # 转换成毫秒
        global total_time
        total_time=execution_time
        return result
    return wrapper

In [4]:
# Optimized addition circuit. "k" is encoded directly into the rotation gates
# to reduce the number of gates and time taken to add the number "k" to the
# specified accumulator.

def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

In [5]:
# Optimized multiplication circuit using optimized addition circuits.
# Multiplication is implemented through repeated addition using an accumulator.

def multiplication(k,wires_m, wires_solution):
    for i in range(len(wires_m)-1,-1,-1):
        qml.ctrl(add_k_fourier, control=wires_m[i])(k, wires_solution[len(wires_m)-1-i:])

@calculate_time_mul
@qml.qnode(dev)
def mul(m,k):
    qml.BasisEmbedding(m, wires=wires_m)  # m encoding
    qml.QFT(wires=wires_solution)
    multiplication(k,wires_m, wires_solution)
    qml.adjoint(qml.QFT)(wires=wires_solution)
    return qml.sample(wires=wires_solution)

In [6]:
@calculate_time_add
@qml.qnode(dev)
def  sum_list(m):
    qml.QFT(wires=range(len(wires_solution)))  # step 1
    for i in range(len(m)):
        add_k_fourier(m[i], wires=range(len(wires_solution)))
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # step 3
    return qml.sample(wires=range(len(wires_solution)))

In [7]:
# Global variables defining the two matrices to be multiplied together. "k"
# represents the size of the matrices (e.g. k = 2 represents 2x2 matrices) and
# val represents the value to fill the matrices with (e.g. val = 2 corresponds
# to matrices where every entry is "2"). The program will perform:
# matrix_a * matrix_b = matrix_c (result is stored in matrix_c)

k=2
val=2
matrix_a=np.full((k,k),val)
matrix_b=np.full((k,k),val)

n=matrix_a.shape[0]
m=matrix_b.shape[1]
matrix_c=np.zeros((n,m))

In [10]:
# Take the dot product between two vectors. Taking the dot product of every
# row of matrix_a with every column of matrix_c allows for the construction of
# matrix_c, their product.
def dot(a,b):
    listx=[]
    for i in range(a.shape[0]):
        mulnum=mul(int(a[i]),int(b[i]))
        mulnum=int(''.join(map(str, list(mulnum))), 2)
        listx.append(mulnum)
    temp_res=sum_list(listx)
    temp_res=int(''.join(map(str, list(temp_res))), 2)
    return temp_res


@calculate_time
def matrix_multiplication_addpro_mulpro():
    # For every row of matrix_a
    for i in range(n):
        # For every column of matrix_b
        for j in range(m):
            matrix_c[i, j] = dot(matrix_a[i], matrix_b[:, j])
            # print(matrix_c[i,j])
            print(matrix_c)

In [11]:
if __name__=='__main__':
    matrix_multiplication_addpro_mulpro()
    # Add a new entry in the CSV file displaying the addition, multiplication,
    # and total time to multiply matrix_a and matrix_b.
    csv_file = open('multication_addpro_mulpro.csv', 'a', newline='')
    writer = csv.writer(csv_file)
    data = [['size', 'value', 'add_time', 'mul_time', 'total_time'], [k, val, add_time, mul_time, total_time]]
    writer.writerows(data)
    csv_file.close()

[[8. 0.]
 [0. 0.]]
[[8. 8.]
 [0. 0.]]
[[8. 8.]
 [8. 0.]]
[[8. 8.]
 [8. 8.]]
